# Instalación de librerías

Lo primero es instalar las librerías necesarias para realizar la tarea.

In [1]:
# Instalamos las librerías necesarias
%pip install scrapy bs4 elasticsearch==8.12.1


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Compilar datos del documento web

Se va a crear un Crawler para extraer información de la página de Bulbapedia. Conretamente, el crawler va a extraer la información de todos lo Pokémon, empezará por un primer enlace (Abomasnow) y seguirá con los enlaces de la página para obtener la información del resto. 

Vamos a crear este crawler en un archivo `.py` a parte ([`crawler.py`](crawler.py#L15)).

Una vez tenemos creado el crawler, vamos a ejecutarlo para obtener los datos. Los resultados de cada pokemon se guardarán en un archivo JSON cuyo nombre será el número de la pokédex del pokemon.

In [1]:
from crawler import PokedexSpyder
import scrapy, os
from scrapy.crawler import CrawlerProcess
import nest_asyncio
nest_asyncio.apply()


# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Comprobamos que existe la carpeta y si no existe la creamos
if not os.path.exists('pokedex'):
    os.mkdir('pokedex')

# Creamos el proceso con el RSSSpider
process.crawl(PokedexSpyder)

# Ejecutamos el Crawler
process.start()

Nidorino -> https://pkparaiso.com//pokedex/nidoran♂.php
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#ventajas
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#habilidades
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#objetos
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#evolucion
Nidorino -> https://pkparaiso.com//pokedex/nidoran♂.php
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php
Nidorino -> https://pkparaiso.com//pokedex/nidoking.php
Nidorino -> https://pkparaiso.com//pokedex/nidoran♂.php
Nidorino -> https://pkparaiso.com//pokedex/nidoking.php
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#descripcion
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#otros
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#estadisticas
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#maximos
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#pokeathlon
Nidorino -> https://pkparaiso.com//pokedex/nidorino.php#localizacion1
N

# Buscador

Importamos las librerías necesarias.

In [2]:
from elasticsearch import Elasticsearch, helpers
import uuid

Nos conectamos a Elasticsearch.

In [3]:
from elasticsearch import Elasticsearch

client = Elasticsearch(['http://localhost:9200'])

def obtener_informacion_cluster():
    try:
        info_cluster = client.cluster.health()
        print("Información del clúster:")
        print(info_cluster)
    except Exception as e:
        print("Error al obtener información del clúster:", e)

obtener_informacion_cluster()

Información del clúster:
{'cluster_name': 'docker-cluster', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 26, 'active_shards': 26, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 1, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 96.29629629629629}


Creamos el índice.

In [4]:
# Nos conectamos a Elasticsearch
es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)

# Nombramos el índice
index_name = 'pokemon_index'

# Borramos el índice si ya existe
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Creamos el índice
es.indices.create(
    index=index_name,
    body={
        "mappings": {
            "properties": {
                "name": {"type": "keyword"},
                "number": {"type": "integer"},
                "url": {"type": "text"},
                "date":        { 
                    "type":   "date",
                    "format": "dd-MM-yyyy"
                },
                "descriptions": {
                    "type":    "object",
                    "dynamic": True
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'pokemon_index'})

Cargamos los datos en Elasticsearch.

In [8]:
import json
import os
from pathlib import Path
from pprint import pprint

# Creamos una lista para guardar los documentos
documents: list = [] 

# Directorio donde se encuentran los archivos JSON
directory: Path = Path('pokedex')

# Creamos un bucle que recorra los archivos JSON de la carpeta bulbapedia
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        file_path = directory / filename
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            document = {
                "_index": index_name,
                "_id": uuid.uuid4(),
                "_source": {
                    "url": data["url"],
                    "number": data["number"],
                    "name": data["name"],
                    "date": data["date"],
                    "descriptions": data["descriptions"],
                },
            }
            documents.append(document)

# Insertamos los documentos en Elasticsearch
helpers.bulk(es, documents)

(501, [])

In [10]:
import time

# Comprobamos que se han insertado los documentos
time.sleep(1)
count = es.count(index=index_name)['count']
print(f"Number of documents in index '{index_name}': {count}")

# Visualizamos un documento
response = es.search(index=index_name, body={"query": {"match_all": {}}})
print("Primer documento:")
pprint(response['hits']['hits'][0]['_source'])

Number of documents in index 'pokemon_index': 501
Primer documento:
{'date': '19-09-2005',
 'descriptions': {' Pokémon Blanco': 'Si están en solitario, no pasa nada; '
                                     'pero si se juntan dos o más, se dice que '
                                     'surge un extraño poder.',
                  ' Pokémon Cristal': 'Se dice que, como hay muchos tipos de '
                                      'Unown, deben de tener numerosas y '
                                      'variadas habilidades.',
                  ' Pokémon Diamante': 'Parecen comunicarse entre ellos '
                                       'telepáticamente. Siempre están pegados '
                                       'a las paredes.',
                  ' Pokémon Esmeralda': 'Tienen forma de caracteres antiguos. '
                                        'No se sabe qué surgió primero: la '
                                        'escritura o los distintos Unown, pero '
                    